In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('Data/Jamestown_weather_2002-current.csv', index_col="Date", parse_dates=True)

In [8]:
df.head()

,MaxTemp,MaxTempMissing,MaxTempEstimated,MinTemp,MinTempMissing,MinTempEstimated,AvgTemp,AvgTempMissing,AvgTempEstimated,AvgBareSoilTemp_F,...,AvgPenmanPETEstimated,TotalRainfall_inch,TotalRainfallMissing,TotalRainfallEstimated,AvgDewPoint_F,AvgDewPointMissing,AvgDewPointEstimated,AvgWindChill_F,AvgWindChillMissing,AvgWindChillEstimated
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,27.303,0,0,9.467,0,0,18.385,0,0,27.933,...,0,NaN,7,0,13.858,0,0,7.459,0,0
2002-01-08,39.815,0,0,25.948,0,0,32.882,0,0,31.238,...,0,NaN,7,0,25.456,0,0,23.226,0,0
2002-01-15,23.837,0,0,2.848,0,0,13.342,0,0,27.270,...,0,NaN,7,0,8.138,0,0,1.728,0,0
2002-01-22,21.085,0,0,3.847,0,0,12.466,0,0,24.517,...,0,NaN,7,0,6.820,0,0,-0.195,0,0
2002-01-29,19.566,0,0,-6.088,0,0,6.739,0,0,23.631,...,0,NaN,7,0,1.558,0,0,-3.079,0,0


In [9]:
df.describe()

,MaxTemp,MaxTempMissing,MaxTempEstimated,MinTemp,MinTempMissing,MinTempEstimated,AvgTemp,AvgTempMissing,AvgTempEstimated,AvgBareSoilTemp_F,...,AvgPenmanPETEstimated,TotalRainfall_inch,TotalRainfallMissing,TotalRainfallEstimated,AvgDewPoint_F,AvgDewPointMissing,AvgDewPointEstimated,AvgWindChill_F,AvgWindChillMissing,AvgWindChillEstimated
count,1145.000000,1145.0,1145.000000,1145.000000,1145.0,1145.000000,1145.000000,1145.0,1145.000000,1145.000000,...,1145.000000,709.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000
mean,52.011427,0.0,0.029694,30.617717,0.0,0.039301,41.314720,0.0,0.040175,47.079124,...,0.001747,0.538072,2.779913,0.047162,32.574510,0.000873,0.053275,35.287160,0.000873,0.120524
std,23.982678,0.0,0.319788,21.321530,0.0,0.343816,22.536766,0.0,0.344984,17.618752,...,0.041776,0.807818,3.383234,0.425841,20.498662,0.029553,0.376019,28.037747,0.029553,0.592322
min,-6.841000,0.0,0.000000,-24.039000,0.0,0.000000,-15.440000,0.0,0.000000,17.738000,...,0.000000,0.000000,0.000000,0.000000,-23.987000,0.000000,0.000000,-34.988000,0.000000,0.000000
25%,31.172000,0.0,0.000000,14.233000,0.0,0.000000,23.063000,0.0,0.000000,31.019000,...,0.000000,0.024000,0.000000,0.000000,17.918000,0.000000,0.000000,12.845000,0.000000,0.000000
50%,55.395000,0.0,0.000000,31.627000,0.0,0.000000,43.737000,0.0,0.000000,43.522000,...,0.000000,0.223000,0.000000,0.000000,32.173000,0.000000,0.000000,38.237000,0.000000,0.000000
75%,74.249000,0.0,0.000000,50.753000,0.0,0.000000,62.393000,0.0,0.000000,64.262000,...,0.000000,0.741000,7.000000,0.000000,52.095000,0.000000,0.000000,62.088000,0.000000,0.000000
max,94.946000,0.0,7.000000,64.421000,0.0,7.000000,77.270000,0.0,7.000000,81.941000,...,1.000000,8.004000,7.000000,7.000000,67.721000,1.000000,7.000000,77.464000,1.000000,7.000000


In [12]:
df.value_counts()

MaxTemp  MaxTempMissing  MaxTempEstimated  MinTemp  MinTempMissing  MinTempEstimated  AvgTemp  AvgTempMissing  AvgTempEstimated  AvgBareSoilTemp_F  AvgBareSoilTempMissing  AvgBareSoilTempEstimated  AvgTurfSoilTemp_F  AvgTurfSoilTempMissing  AvgTurfSoilTempEstimated  AvgWindSpeed  AvgWindSpeedMissing  AvgWindSpeedEstimated  MaxWindSpeed  MaxWindSpeedMissing  MaxWindSpeedEstimated  TotalSolarRad_Lys  TotalSolarRadMissing  TotalSolarRadEstimated  TotalPenmanPET_inch  TotalPenmanPETMissing  TotalPenmanPETEstimated  AvgPenmanPET_inch  AvgPenmanPETMissing  AvgPenmanPETEstimated  TotalRainfall_inch  TotalRainfallMissing  TotalRainfallEstimated  AvgDewPoint_F  AvgDewPointMissing  AvgDewPointEstimated  AvgWindChill_F  AvgWindChillMissing  AvgWindChillEstimated
7.799    0               0                 -11.035  0               0                 -1.618   0               0                 19.714             0                       0                         25.361             0                    